In [1]:
import os
import time
import argparse
import numpy as np
from PIL import Image
from glob import glob
from ntpath import basename
from os.path import join, exists
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torchvision.utils import save_image
import torchvision.transforms as transforms

In [2]:
class Options:
    data_dir = "data/A/1.jpg"
    sample_dir = "data/output/"
    model_name = "funiegan"  # or "ugan"
    model_path = "checkpoints/FunieGAN/paired/generator_60.pth"

opt = Options()

In [3]:
assert exists(opt.model_path), "model not found"
os.makedirs(opt.sample_dir, exist_ok=True)
is_cuda = torch.cuda.is_available()
Tensor = torch.cuda.FloatTensor if is_cuda else torch.FloatTensor 
print(is_cuda)

True


In [4]:
if opt.model_name.lower()=='funiegan':
    from nets import funiegan
    model = funiegan.GeneratorFunieGAN()
else:
    pass

In [5]:
model.load_state_dict(torch.load(opt.model_path))
if is_cuda: model.cuda()
model.eval()
print ("Loaded model from %s" % (opt.model_path))

Loaded model from checkpoints/FunieGAN/paired/generator_60.pth


In [6]:
img_width, img_height, channels = 1024, 1024, 3
transforms_ = [transforms.Resize((img_height, img_width), Image.BICUBIC),
               transforms.ToTensor(),
               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),]
transform = transforms.Compose(transforms_)

In [7]:
print(f"Model loaded: {opt.model_name}")
print(f"Input folder: {opt.data_dir}")
print(f"Output folder: {opt.sample_dir}")
print(f"Looking for test images...")

test_files = sorted(glob(join(opt.data_dir, "*")))
print(f"Found {len(test_files)} images.\n")

Model loaded: funiegan
Input folder: data/A/1.jpg
Output folder: data/output/
Looking for test images...
Found 0 images.



In [8]:
device = "cuda" if torch.cuda.is_available else "cpu"

In [9]:
from PIL import Image
from torchvision import transforms
from torch.autograd import Variable
from torchvision.utils import save_image
import torch
import time
from os.path import join, basename

img_width, img_height = 256, 256  # Match training size
transforms_ = [
    transforms.Resize((img_height, img_width), Image.BICUBIC),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
]
transform = transforms.Compose(transforms_)

input_image_path = "data/A/img.jpg"

try:
    print(f"Processing {input_image_path}")
    inp_img = transform(Image.open(input_image_path).convert("RGB"))
    inp_img = Variable(inp_img).unsqueeze(0).type(torch.FloatTensor).to(device)  # device = 'cuda' or 'cpu'

    s = time.time()
    with torch.no_grad():
        gen_img = model(inp_img)
    elapsed = time.time() - s
    print(f"Inference time: {elapsed:.3f} sec")

    # Save side-by-side input/output
    img_sample = torch.cat((inp_img.data, gen_img.data), -1)
    save_path = join(opt.sample_dir, basename(input_image_path))
    save_image(img_sample, save_path, normalize=True)
    print(f"Saved output to: {save_path}")

except Exception as e:
    print(f"Error processing image: {e}")


Processing data/A/img.jpg
Inference time: 0.296 sec
Saved output to: data/output/img.jpg


In [58]:
import cv2
import numpy as np
from PIL import Image
from torchvision import transforms
from torch.autograd import Variable
from torchvision.utils import save_image
import torch
import os
import time
from os.path import join, basename

# ----- Config -----
input_image_path = "data/A/1.jpg"  # Your test image
output_dir = "output"
os.makedirs(output_dir, exist_ok=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
img_width, img_height = 256, 256

# ----- Preprocessing -----
transform = transforms.Compose([
    transforms.Resize((img_height, img_width), Image.BICUBIC),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

# ----- Load and Run FUNIE-GAN -----
inp_img = transform(Image.open(input_image_path).convert("RGB"))
inp_img = Variable(inp_img).unsqueeze(0).to(device)

with torch.no_grad():
    start = time.time()
    gen_img = model(inp_img)
    print(f"Inference time: {time.time() - start:.3f} sec")

# Save raw output
gen_path = join(output_dir, "funie_output.jpg")
save_image(gen_img.data, gen_path, normalize=True)
print(f"Saved FUNIE-GAN output to: {gen_path}")

# ----- Post-processing -----
def white_balance(img):
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    avg_a, avg_b = np.mean(a), np.mean(b)
    
    a = a.astype(np.float32)
    b = b.astype(np.float32)
    l_scaled = l.astype(np.float32) / 255.0

    a = a - ((avg_a - 128) * l_scaled * 1.1)
    b = b - ((avg_b - 128) * l_scaled * 1.1)

    a = np.clip(a, 0, 255).astype(np.uint8)
    b = np.clip(b, 0, 255).astype(np.uint8)

    lab = cv2.merge([l, a, b])
    return cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)

def clahe_contrast(img):
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    l = clahe.apply(l)
    lab = cv2.merge((l, a, b))
    return cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)

def gamma_correction(img, gamma=1.2):
    inv_gamma = 1.0 / gamma
    table = np.array([(i / 255.0) ** inv_gamma * 255 for i in range(256)]).astype("uint8")
    return cv2.LUT(img, table)

# Load GAN output as BGR
gan_img = cv2.imread(gen_path)
wb_img = white_balance(gan_img)
clahe_img = clahe_contrast(wb_img)
final_img = gamma_correction(clahe_img, gamma=1.1)

# Save final enhanced result
final_path = join(output_dir, "funie_corrected.jpg")
final_img = np.clip(final_img, 0, 255).astype(np.uint8)
success = cv2.imwrite(final_path, final_img)
if success:
    print(f"✅ Corrected image saved at: {final_path}")
else:
    print(f"❌ Failed to save image at: {final_path}")

# ----- Side-by-side visual -----
orig = cv2.resize(cv2.imread(input_image_path), (256, 256))
gan = cv2.resize(gan_img, (256, 256))
post = cv2.resize(final_img, (256, 256))
comparison = cv2.hconcat([orig, gan, post])
cv2.imwrite(join(output_dir, "comparison.jpg"), comparison)
print("Saved comparison image.")


Inference time: 0.004 sec
Saved FUNIE-GAN output to: output\funie_output.jpg
✅ Corrected image saved at: output\funie_corrected.jpg
Saved comparison image.


In [ ]:
import cv2

input_video_path = "data/A/vidd.mp4"  # <-- your video input
output_video_path = join(opt.sample_dir, "enhanced_" + basename(input_video_path))

try:
    print(f"Processing video: {input_video_path}")

    # Open video capture
    cap = cv2.VideoCapture(input_video_path)
    if not cap.isOpened():
        raise IOError(f"Cannot open video: {input_video_path}")

    # Get video properties
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Output writer: double width for side-by-side display
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width * 2, height))

    times = []
    frame_count = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Resize + convert to PIL for model input
        pil_img = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)).resize((256, 256), Image.BICUBIC)
        inp_tensor = transform(pil_img).unsqueeze(0).type(Tensor)

        # Inference
        s = time.time()
        with torch.no_grad():
            gen_tensor = model(inp_tensor)
        times.append(time.time() - s)

        # Convert output to numpy
        inp_np = inp_tensor.squeeze().cpu().numpy().transpose(1, 2, 0)
        gen_np = gen_tensor.squeeze().cpu().numpy().transpose(1, 2, 0)

        # Denormalize
        inp_np = ((inp_np * 0.5 + 0.5) * 255).astype(np.uint8)
        gen_np = ((gen_np * 0.5 + 0.5) * 255).astype(np.uint8)

        # Resize output back to original video size
        inp_resized = cv2.resize(inp_np, (width, height))
        gen_resized = cv2.resize(gen_np, (width, height))

        # Stack side-by-side and convert to BGR for OpenCV
        side_by_side = cv2.hconcat([
            cv2.cvtColor(inp_resized, cv2.COLOR_RGB2BGR),
            cv2.cvtColor(gen_resized, cv2.COLOR_RGB2BGR)
        ])

        out.write(side_by_side)
        frame_count += 1
        print(f"Processed frame {frame_count}")

    # Release everything
    cap.release()
    out.release()

    if len(times) > 1:
        Ttime, Mtime = np.sum(times[1:]), np.mean(times[1:])
        print(f"\nProcessed {frame_count} frames.")
        print(f"Time taken: {Ttime:.2f} sec at {1. / Mtime:.2f} fps")
        print(f"Saved enhanced video to: {output_video_path}")

except Exception as e:
    print(f"Error processing video: {e}")


Processing video: data/A/vid.mp4
Processed frame 1
Processed frame 2
Processed frame 3
Processed frame 4
Processed frame 5
Processed frame 6
Processed frame 7
Processed frame 8
Processed frame 9
Processed frame 10
Processed frame 11
Processed frame 12
Processed frame 13
Processed frame 14
Processed frame 15
Processed frame 16
Processed frame 17
Processed frame 18
Processed frame 19
Processed frame 20
Processed frame 21
Processed frame 22
Processed frame 23
Processed frame 24
Processed frame 25


: 

In [ ]:
import cv2
import time
import numpy as np
from os.path import join, basename
from PIL import Image
from torchvision import transforms
from torch.autograd import Variable
import torch

# === Paths ===
input_video_path = "data/A/vidd2.mp4"
output_video_path = join(opt.sample_dir, "enhancedd_" + basename(input_video_path))

# === Tensor type ===
Tensor = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.FloatTensor

# === Transform (same as model training) ===
transform = transforms.Compose([
    transforms.Resize((256, 256)),  # Adjust based on training size
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])  # Normalize to [-1, 1]
])

# === Model: Ensure it is loaded ===
# model = YourModelClass()
# model.load_state_dict(torch.load("your_model_path.pth"))
# model.eval()
# model.to("cuda" or "cpu")

try:
    print(f"Processing video: {input_video_path}")
    cap = cv2.VideoCapture(input_video_path)
    if not cap.isOpened():
        raise IOError(f"Cannot open video: {input_video_path}")

    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    max_frames = int(fps * 4)

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width * 2, height))

    times = []
    frame_count = 0

    while frame_count < max_frames:
        ret, frame = cap.read()
        if not ret:
            break

        # Convert frame to tensor
        pil_img = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        inp_tensor = transform(pil_img).unsqueeze(0).type(Tensor)
        inp_tensor = Variable(inp_tensor)

        # === Model Inference ===
        s = time.time()
        with torch.no_grad():
            gen_tensor = model(inp_tensor)
        times.append(time.time() - s)

        # === Postprocessing ===
        gen_np = gen_tensor.squeeze().cpu().numpy().transpose(1, 2, 0)
        gen_np = ((gen_np * 0.5 + 0.5) * 255).astype(np.uint8)  # De-normalize

        # === Apply Gaussian Blur ===
        gen_np = cv2.GaussianBlur(gen_np, (5, 5), sigmaX=1.0)

        # === Apply Sharpening Filter ===
        sharpen_kernel = np.array([[0, -1, 0],
                                   [-1, 5, -1],
                                   [0, -1, 0]])
        gen_np = cv2.filter2D(gen_np, -1, sharpen_kernel)

        # === Resize for final video output ===
        inp_np = inp_tensor.squeeze().cpu().numpy().transpose(1, 2, 0)
        inp_np = ((inp_np * 0.5 + 0.5) * 255).astype(np.uint8)
        inp_np = cv2.resize(inp_np, (width, height))
        gen_np = cv2.resize(gen_np, (width, height))

        # === Concatenate side-by-side ===
        side_by_side = cv2.hconcat([
            cv2.cvtColor(inp_np, cv2.COLOR_RGB2BGR),
            cv2.cvtColor(gen_np, cv2.COLOR_RGB2BGR)
        ])
        out.write(side_by_side)
        frame_count += 1
        print(f"Processed frame {frame_count}/{max_frames}")

    cap.release()
    out.release()

    if len(times) > 0:
        Ttime, Mtime = np.sum(times), np.mean(times)
        print(f"\nProcessed {frame_count} frames.")
        print(f"Time taken: {Ttime:.2f} sec at {1. / Mtime:.2f} fps")
        print(f"Saved enhanced video to: {output_video_path}")

except Exception as e:
    print(f"Error processing video: {e}")


Processing video: data/A/vidd2.mp4
Processed frame 1/128
Processed frame 2/128
Processed frame 3/128
Processed frame 4/128
Processed frame 5/128
Processed frame 6/128
Processed frame 7/128
Processed frame 8/128
Processed frame 9/128
Processed frame 10/128
Processed frame 11/128
Processed frame 12/128
Processed frame 13/128
Processed frame 14/128
Processed frame 15/128
Processed frame 16/128
Processed frame 17/128
Processed frame 18/128
Processed frame 19/128
Processed frame 20/128
Processed frame 21/128
Processed frame 22/128
Processed frame 23/128
Processed frame 24/128
Processed frame 25/128
Processed frame 26/128
Processed frame 27/128
Processed frame 28/128
Processed frame 29/128
Processed frame 30/128
Processed frame 31/128
Processed frame 32/128
Processed frame 33/128
Processed frame 34/128
Processed frame 35/128
Processed frame 36/128
Processed frame 37/128
Processed frame 38/128
Processed frame 39/128
Processed frame 40/128
Processed frame 41/128
Processed frame 42/128
Processe

In [11]:
import cv2
import time
import numpy as np
from os.path import join, basename
from PIL import Image
from torchvision import transforms
from torch.autograd import Variable
import torch

# === POST-PROCESSING FUNCTIONS ===

def white_balance_safe(img):
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    avg_a = np.mean(a)
    avg_b = np.mean(b)
    l_scaled = l.astype(np.float32) / 255.0
    a = np.clip(a.astype(np.float32) - ((avg_a - 128) * l_scaled * 0.6), 0, 255)
    b = np.clip(b.astype(np.float32) - ((avg_b - 128) * l_scaled * 0.6), 0, 255)
    lab = cv2.merge([l, a.astype(np.uint8), b.astype(np.uint8)])
    return cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)

def clahe_contrast_safe(img):
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=1.5, tileGridSize=(8, 8))
    cl = clahe.apply(l)
    lab = cv2.merge((cl, a, b))
    return cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)

def gamma_correction_safe(img, gamma=1.05):
    invGamma = 1.0 / gamma
    table = np.array([(i / 255.0) ** invGamma * 255 for i in range(256)]).astype("uint8")
    return cv2.LUT(img, table)

def denoise_if_needed(img):
    return cv2.fastNlMeansDenoisingColored(img, None, 10, 10, 7, 21)


# === MAIN VIDEO ENHANCEMENT CODE ===

# Inputs
input_video_path = "data/A/vidd2.mp4"
output_video_path = join(opt.sample_dir, "enhancedd_" + basename(input_video_path))

# Tensor type
Tensor = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.FloatTensor

# Transform
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

try:
    print(f"Processing video: {input_video_path}")

    cap = cv2.VideoCapture(input_video_path)
    if not cap.isOpened():
        raise IOError(f"Cannot open video: {input_video_path}")

    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    max_frames = int(fps * 4)  # limit to ~50 seconds

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width * 2, height))

    frame_count = 0
    single_frame_time = None

    while frame_count < max_frames:
        ret, frame = cap.read()
        if not ret:
            break

        # Convert to PIL
        pil_img = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        inp_tensor = transform(pil_img).unsqueeze(0)
        inp_tensor = Variable(inp_tensor).type(Tensor)

        with torch.no_grad():
            if frame_count == max_frames - 1:
                start = time.time()
            gen_tensor = model(inp_tensor)
            if frame_count == max_frames - 1:
                torch.cuda.synchronize()
                single_frame_time = time.time() - start

        # Post-process the tensor
        gen_tensor = torch.nan_to_num(gen_tensor, nan=0.0, posinf=1.0, neginf=-1.0)

        inp_np = inp_tensor.squeeze().cpu().numpy().transpose(1, 2, 0)
        gen_np = gen_tensor.squeeze().cpu().numpy().transpose(1, 2, 0)

        inp_np = ((inp_np * 0.5 + 0.5) * 255).astype(np.uint8)
        gen_np = ((gen_np * 0.5 + 0.5) * 255).astype(np.uint8)

        inp_resized = cv2.resize(inp_np, (width, height))
        gen_resized = cv2.resize(gen_np, (width, height))

        # BGR conversion
        gen_bgr = cv2.cvtColor(gen_resized, cv2.COLOR_RGB2BGR)

        # Safe post-processing
        denoised = denoise_if_needed(gen_bgr)
        wb_img = white_balance_safe(denoised)
        clahe_img = clahe_contrast_safe(wb_img)
        final_img = gamma_correction_safe(clahe_img, gamma=1.05)

        side_by_side = cv2.hconcat([
            cv2.cvtColor(inp_resized, cv2.COLOR_RGB2BGR),
            final_img
        ])
        out.write(side_by_side)

        frame_count += 1
        print(f"Processed frame {frame_count}/{max_frames}")

    cap.release()
    out.release()

    print(f"\nProcessed {frame_count} frames.")
    print(f"Saved enhanced video to: {output_video_path}")
    if single_frame_time:
        print(f"Model inference time for last frame: {single_frame_time:.4f} sec ({1. / single_frame_time:.2f} fps)")

except Exception as e:
    print(f"Error processing video: {e}")



Processing video: data/A/vidd2.mp4
Processed frame 1/128
Processed frame 2/128
Processed frame 3/128
Processed frame 4/128
Processed frame 5/128
Processed frame 6/128
Processed frame 7/128
Processed frame 8/128
Processed frame 9/128
Processed frame 10/128
Processed frame 11/128
Processed frame 12/128
Processed frame 13/128
Processed frame 14/128
Processed frame 15/128
Processed frame 16/128
Processed frame 17/128
Processed frame 18/128
Processed frame 19/128
Processed frame 20/128
Processed frame 21/128
Processed frame 22/128
Processed frame 23/128
Processed frame 24/128
Processed frame 25/128
Processed frame 26/128
Processed frame 27/128
Processed frame 28/128
Processed frame 29/128
Processed frame 30/128
Processed frame 31/128
Processed frame 32/128
Processed frame 33/128
Processed frame 34/128
Processed frame 35/128
Processed frame 36/128
Processed frame 37/128
Processed frame 38/128
Processed frame 39/128
Processed frame 40/128
Processed frame 41/128
Processed frame 42/128
Processe

In [10]:
import cv2
import time
import numpy as np
from PIL import Image
import torch
from torchvision import transforms
from torch.autograd import Variable

# Model preparation
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Transform (adjust to match training settings)
transform = transforms.Compose([
    transforms.Resize((512, 512)),  # or your model's input size
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
Tensor = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.FloatTensor

# Open webcam
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise IOError("Cannot open webcam")

print("Press 'q' to quit.")

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame.")
        break

    # Preprocess frame
    pil_img = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    inp_tensor = transform(pil_img).unsqueeze(0).to(device)

    # Inference timing
    start = time.time()
    with torch.no_grad():
        gen_tensor = model(inp_tensor)
    torch.cuda.synchronize()
    infer_time = time.time() - start

    # Postprocess output
    inp_np = inp_tensor.squeeze().cpu().numpy().transpose(1, 2, 0)
    gen_np = gen_tensor.squeeze().cpu().numpy().transpose(1, 2, 0)
    inp_np = ((inp_np * 0.5 + 0.5) * 255).astype(np.uint8)
    gen_np = ((gen_np * 0.5 + 0.5) * 255).astype(np.uint8)

    # Resize to webcam resolution
    height, width = frame.shape[:2]
    inp_resized = cv2.resize(inp_np, (width, height))
    gen_resized = cv2.resize(gen_np, (width, height))

    # Combine and show
    combined = cv2.hconcat([
        cv2.cvtColor(inp_resized, cv2.COLOR_RGB2BGR),
        cv2.cvtColor(gen_resized, cv2.COLOR_RGB2BGR)
    ])
    fps = 1. / infer_time
    cv2.putText(combined, f"{infer_time:.3f}s ({fps:.2f} FPS)", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 0), 2)

    cv2.imshow("Original | Enhanced", combined)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Press 'q' to quit.


In [21]:
times = []
test_files = sorted(glob(join(opt.data_dir, "*.*")))
for path in test_files:
    try:
        print(f"Processing {path}")
        inp_img = transform(Image.open(path).convert("RGB"))
        inp_img = Variable(inp_img).type(Tensor).unsqueeze(0)
        s = time.time()
        gen_img = model(inp_img)
        times.append(time.time()-s)
        img_sample = torch.cat((inp_img.data, gen_img.data), -1)
        save_image(img_sample, join(opt.sample_dir, basename(path)), normalize=True)
        print ("Tested: %s" % path)
    except Exception as e:
        print(f"Error processing {path}: {e}")


## run-time    
if (len(times) > 1):
    print ("\nTotal samples: %d" % len(test_files)) 
    # accumulate frame processing times (without bootstrap)
    Ttime, Mtime = np.sum(times[1:]), np.mean(times[1:]) 
    print ("Time taken: %d sec at %0.3f fps" %(Ttime, 1./Mtime))
    print("Saved generated images in in %s\n" %(opt.sample_dir))